In [32]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier

Читаем входные файлы с данными

In [33]:
transactions = pd.read_csv('data/transactions.csv')
customers_gender = pd.read_csv('data/customers_gender_train.csv')

Для каждого клиента подсчитываем различные mcc_code

In [34]:
X = transactions.groupby('customer_id') \
                    .apply(lambda x: x[['mcc_code']].unstack().value_counts()) \
                    .unstack() \
                    .fillna(0)

In [35]:
customers_gender = customers_gender.set_index('customer_id')

In [36]:
Y_train = customers_gender.loc[X.index].gender
Y_train = Y_train.reset_index()
del Y_train['customer_id']
Y_train = Y_train.dropna(0)

In [37]:
X_train = X.reset_index()
X_train = X_train.loc[Y_train.index].set_index('customer_id')

Обучаемся на всех данных

In [38]:
clf = GradientBoostingClassifier(random_state=13)
clf.fit(X_train, Y_train.values[:, 0])

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=13,
              subsample=1.0, verbose=0, warm_start=False)

Предсказываем пол для тестовых данных и создаём файл с ответом

In [39]:
X_test = X.drop(customers_gender.index)
result = pd.DataFrame(X_test.index, columns=['customer_id'])
result['gender'] = clf.predict_proba(X_test)[:, 1]

In [40]:
result.to_csv('baseline_a.csv', index=False)

In [10]:
import xgboost as xgb
from sklearn.grid_search import GridSearchCV

/home/laol/shad-env/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/laol/shad-env/local/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [21]:
cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
ind_params = {'learning_rate': 0.1, 'n_estimators': 1000, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic'}
optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                            scoring = 'accuracy', cv = 5, n_jobs = -1, verbose = 10) 

In [13]:
dtrain = xgb.DMatrix(X_train, label=Y_train.values[:, 0])


clf = xgb.train(param, dtrain)

In [49]:
#dtest = xgb.DMatrix(X.drop(customers_gender.index), label=targetTest)
#preds = clf.predict(dtest)

In [39]:
X_test = X.drop(customers_gender.index)
result = pd.DataFrame(X_test.index, columns=['customer_id'])
result['gender'] = clf.predict_proba(X_test)[:, 1]

array([ 0.,  1.,  1., ...,  1.,  1.,  1.])

In [26]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.grid_search import GridSearchCV

/home/laol/shad-env/local/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [ ]:
cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
ind_params = {'learning_rate': 0.1, 'n_estimators': 1000, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic'}
optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                            scoring = 'accuracy', cv = 5, n_jobs = -1, verbose = 10) 

In [ ]:
optimized_GBM.fit(X_train, Y_train.values[:, 0])

In [42]:

param = {'max_depth':7, 
         'eta':0.01, 
         'objective':'binary:logistic', 
         'eval_metric': 'auc'}


In [55]:
X = X_train
Y = Y_train
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, Y, test_size=0.25, random_state=1)
model = XGBClassifier(n_estimators=3000, max_depth=4, learning_rate=0.01)
eval_set = [(X_test2, y_test2)]
model.fit(X_train2, y_train2, eval_metric="auc", eval_set=eval_set, early_stopping_rounds=10)

[0]	validation_0-auc:0.745989
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.751743
[2]	validation_0-auc:0.752753
[3]	validation_0-auc:0.753024
[4]	validation_0-auc:0.753165
[5]	validation_0-auc:0.758036
[6]	validation_0-auc:0.758808
[7]	validation_0-auc:0.760678
[8]	validation_0-auc:0.760857
[9]	validation_0-auc:0.762383
[10]	validation_0-auc:0.762333
[11]	validation_0-auc:0.764042
[12]	validation_0-auc:0.764773
[13]	validation_0-auc:0.764872
[14]	validation_0-auc:0.76498
[15]	validation_0-auc:0.764754
[16]	validation_0-auc:0.765394
[17]	validation_0-auc:0.765859
[18]	validation_0-auc:0.766139
[19]	validation_0-auc:0.765888
[20]	validation_0-auc:0.76715
[21]	validation_0-auc:0.767452
[22]	validation_0-auc:0.767584
[23]	validation_0-auc:0.76719
[24]	validation_0-auc:0.768176
[25]	validation_0-auc:0.76833
[26]	validation_0-auc:0.768039
[27]	validation_0-auc:0.768342
[28]	validation_0-auc:0.76799
[29]	validation_0-auc:0.768812
[30]	validation_0-auc

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=3000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [47]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

result['gender'] = model.predict_proba(X_test)[:, 1]
result.to_csv('baseline_b.csv', index=False)

In [61]:
import xgboost as xgb

In [ ]:
gbm = xgb.XGBClassifier()
gbm_params = {
    'learning_rate': [0.05, 0.1],
    'n_estimators': [300, 1000],
    'max_depth': [2, 3, 10],
}
cv = StratifiedKFold(X)
grid = GridSearchCV(gbm, gbm_params,scoring='roc_auc',cv=cv,verbose=10,n_jobs=-1)
grid.fit(train_X, train_y)

In [74]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
    min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
    objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
    param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

X_train2, X_test2, y_train2, y_test2 = train_test_split(X, Y, test_size=0.25, random_state=1)

gsearch1.fit(X_train2, y_train2)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

IndexError: too many indices for array

In [ ]:
# This script shows you how to make a submission using a few
# useful Python libraries.
# It gets a public leaderboard score of 0.76077.
# Maybe you can tweak it and do better...?

import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import auc

# Load the data
train_df = pd.read_csv('data/tit/train.csv', header=0)
test_df = pd.read_csv('data/tit/test.csv', header=0)

# We'll impute missing values using the median for numeric columns and the most
# common value for string columns.
# This is based on some nice code by 'sveitser' at http://stackoverflow.com/a/25562948
from sklearn.base import TransformerMixin
class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)

feature_columns_to_use = ['Pclass','Sex','Age','Fare','Parch']
nonnumeric_columns = ['Sex']

# Join the features from train and test together before imputing missing values,
# in case their distribution is slightly different
big_X = train_df[feature_columns_to_use].append(test_df[feature_columns_to_use])
big_X_imputed = DataFrameImputer().fit_transform(big_X)

# XGBoost doesn't (yet) handle categorical features automatically, so we need to change
# them to columns of integer values.
# See http://scikit-learn.org/stable/modules/preprocessing.html#preprocessing for more
# details and options
le = LabelEncoder()
for feature in nonnumeric_columns:
    big_X_imputed[feature] = le.fit_transform(big_X_imputed[feature])

# Prepare the inputs for the model
train_X = big_X_imputed[0:train_df.shape[0]].as_matrix()
test_X = big_X_imputed[train_df.shape[0]::].as_matrix()
train_y = train_df['Survived']

gbm = xgb.XGBClassifier()
gbm_params = {
    'learning_rate': [0.05, 0.1],
    'n_estimators': [300, 1000],
    'max_depth': [2, 3, 10],
}
cv = StratifiedKFold(train_y)
grid = GridSearchCV(gbm, gbm_params,scoring='roc_auc',cv=cv,verbose=10,n_jobs=-1)
grid.fit(train_X, train_y)

print (grid.best_params_)

# You can experiment with many other options here, using the same .fit() and .predict()
# methods; see http://scikit-learn.org
# This example uses the current build of XGBoost, from https://github.com/dmlc/xgboost
# gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(train_X, train_y)
predictions = grid.best_estimator_.predict(test_X)

# Kaggle needs the submission to have a certain format;
# see https://www.kaggle.com/c/titanic-gettingStarted/download/gendermodel.csv
# for an example of what it's supposed to look like.
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],
                            'Survived': predictions })
submission.to_csv("submission.csv", index=False)

In [ ]:
print 1